In [2]:
'''
司法院大法官 -> 解釋及不受理決議 -> 不受理決議

範例連結: https://cons.judicial.gov.tw/jcc/zh-tw/rrp04?page=1
'''
import requests, json
from bs4 import BeautifulSoup

# 整理所有資料的變數
listData = []

# 走訪總頁數
pages = 1

# 取得 table 中的列表連結
def getListItems():
    for page in range(1, pages + 1):
        response = requests.get(f"https://cons.judicial.gov.tw/jcc/zh-tw/rrp04?page={page}")
        soup = BeautifulSoup(response.text, 'lxml')
        a_elms = soup.select("table.blocky_body.form_table.form_table_second.sm-responsive tbody tr td[data-head='案號'] a")
        for a in a_elms:
            listData.append({
                "title": a.get_text(),
                "link": "https://cons.judicial.gov.tw" + a["href"]
            })

# 根據先前儲存的列表連結，爬出需要的資訊
def getItemDetail():
    for index, _dict in enumerate(listData):
        response = requests.get(_dict['link'])
        soup = BeautifulSoup(response.text, 'lxml')
        pre_elms = soup.select('div.item.title-w-8 pre.content.pure_text')
        listData[index]['公布院令'] = pre_elms[0].get_text()
        listData[index]['會次'] = pre_elms[1].get_text()
        listData[index]['日期'] = pre_elms[2].get_text()
        listData[index]['案號'] = pre_elms[3].get_text()
        listData[index]['聲請人'] = pre_elms[4].get_text()
        listData[index]['案由'] = pre_elms[5].get_text()
        listData[index]['決議'] = pre_elms[6].get_text()

# 將所有資訊轉成 JSON 檔
def saveJson():
    with open("JCC_reject.json", "w", encoding="utf-8") as file:
        file.write( json.dumps(listData, ensure_ascii=False, indent=4) )

# 主程式區段
if __name__ == "__main__":
    getListItems()
    getItemDetail()
    saveJson()
        